In [ ]:
import pandas as pd
import requests
import numpy as np
import openpyxl
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv
import os

# Carga y descarga de API

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
#url de la API
api_url='https://api.yelp.com/v3/businesses/search'

load_dotenv("credenciales.env")

# Carga variables de entorno
clientid = os.getenv('YELP_CLIENT_ID')
apikey = os.getenv('YELP_API_KEY')

if not clientid or not apikey:
    raise ValueError("Credenciales de Yelp no encontradas en .env")

headers = {'Authorization': 'Bearer %s' % apikey}

In [4]:
ciudad = "New York" # Se puede elegir cualquier ciudad del dataset entregado

In [5]:
# De esta manera se consulta el API de Yelp para obtener información sobre restaurantes en la ciudad especificada
params={'term':'restaurants','location': ciudad,'limit':50}
response=requests.get(api_url,params=params,headers=headers)
data=response.json()
data

{'businesses': [{'id': '8YWLuLUKj0t_0_Xv06UUtw',
   'alias': 'yes-apothecary-new-york-6',
   'name': "Ye's Apothecary",
   'image_url': 'https://s3-media0.fl.yelpcdn.com/bphoto/y4pAeBfhhPy-u8MIlvHhsA/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/yes-apothecary-new-york-6?adjust_creative=GWOCZh9-BmZxtdsAjr7Gug&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=GWOCZh9-BmZxtdsAjr7Gug',
   'review_count': 998,
   'categories': [{'alias': 'szechuan', 'title': 'Szechuan'},
    {'alias': 'speakeasies', 'title': 'Speakeasies'}],
   'rating': 4.6,
   'coordinates': {'latitude': 40.71945, 'longitude': -73.9898},
   'transactions': [],
   'price': '$$',
   'location': {'address1': '119 Orchard St',
    'address2': None,
    'address3': '',
    'city': 'New York',
    'zip_code': '10002',
    'country': 'US',
    'state': 'NY',
    'display_address': ['119 Orchard St', 'New York, NY 10002']},
   'phone': '+19296552515',
   'display_phone': '(929) 655-2515',

In [6]:
#Veamos los keys del diccionario recibido
data.keys()

dict_keys(['businesses', 'total', 'region'])

In [7]:
#El primer elemento del diccionario indica el total de restaurants existentes en la API
print('En total la base de datos registra %d restaurants'%data['total'])

En total la base de datos registra 23800 restaurants


In [8]:
data['businesses'] # Hasta acá se le entregaría a los estudiantes

[{'id': '8YWLuLUKj0t_0_Xv06UUtw',
  'alias': 'yes-apothecary-new-york-6',
  'name': "Ye's Apothecary",
  'image_url': 'https://s3-media0.fl.yelpcdn.com/bphoto/y4pAeBfhhPy-u8MIlvHhsA/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/yes-apothecary-new-york-6?adjust_creative=GWOCZh9-BmZxtdsAjr7Gug&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=GWOCZh9-BmZxtdsAjr7Gug',
  'review_count': 998,
  'categories': [{'alias': 'szechuan', 'title': 'Szechuan'},
   {'alias': 'speakeasies', 'title': 'Speakeasies'}],
  'rating': 4.6,
  'coordinates': {'latitude': 40.71945, 'longitude': -73.9898},
  'transactions': [],
  'price': '$$',
  'location': {'address1': '119 Orchard St',
   'address2': None,
   'address3': '',
   'city': 'New York',
   'zip_code': '10002',
   'country': 'US',
   'state': 'NY',
   'display_address': ['119 Orchard St', 'New York, NY 10002']},
  'phone': '+19296552515',
  'display_phone': '(929) 655-2515',
  'distance': 1591.3798693136603},
 

----

# Transformacion a DataFrame

In [9]:
offset=0

ciudad = 'New York'

allrests=[]

#iteramos hasta llegar a los 200 resultados (el maximo que permite la API)
while offset<=150:  

    # Parametros de la consulta a la api
    params={'term':'restaurants','location': ciudad,'limit':50,'offset':offset}
    
    response=requests.get(api_url,params=params,headers=headers)
    data=response.json()
    
    
    print(len(data['businesses']),'results found at offset', offset)

    data_businesses = data.get('businesses', [])
    if not data_businesses:
        print('Dataframe createf with', len(rests), 'rows')
        break
    #dataframe con 50 resultados
    rests = pd.json_normalize(
        data['businesses'],
        sep='_',
        
        meta=[
            'id', 'alias', 'name', 'image_url', 'is_closed', 'url', 'review_count', 'rating',
            'coordinates', 'transactions', 'price', 'location_address1', 'location_address2',
            'location_address3', 'location_city', 'location_zip_code', 'location_country',
            'location_state', 'location_display_address', 'phone', 'display_phone', 'distance'
        ],
        meta_prefix='business_',
        errors='ignore' 
    )
    print('Dataframe created with', len(rests), 'rows')
    rests['city'] = ciudad #agregamos la ciudad para analisis posterior
        
    # Agregamos este resultado a la lista de dataframes
    allrests.append(rests)

    #avanzamos con 50 registros
    offset=offset+50

50 results found at offset 0
Dataframe created with 50 rows
50 results found at offset 50
Dataframe created with 50 rows
50 results found at offset 100
Dataframe created with 50 rows
50 results found at offset 150
Dataframe created with 50 rows


In [10]:
rests['categories'].apply(lambda x: ', '.join([cat['title'] for cat in x]))

0                       Bakeries, Taiwanese, American
1                                              Korean
2                                        New American
3                 Italian, Diners, Breakfast & Brunch
4                                                Thai
5                        Cafes, Coffee & Tea, Italian
6           Brazilian, Breakfast & Brunch, Acai Bowls
7                                             Italian
8                                               Pizza
9                                             Italian
10              Seafood, Mediterranean, Cocktail Bars
11                                              Cafes
12                 Cajun/Creole, Sushi Bars, Japanese
13                               Ramen, Noodles, Soup
14                         New American, Asian Fusion
15                              Venezuelan, Empanadas
16                                     Cuban, Mexican
17                             Korean, Barbeque, Bars
18                         K

In [11]:
# Finalmente concatenamos todos los dataframes en uno solo
rests=pd.concat(allrests,ignore_index=True)
rests.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        200 non-null    object 
 1   alias                     200 non-null    object 
 2   name                      200 non-null    object 
 3   image_url                 200 non-null    object 
 4   is_closed                 200 non-null    bool   
 5   url                       200 non-null    object 
 6   review_count              200 non-null    int64  
 7   categories                200 non-null    object 
 8   rating                    200 non-null    float64
 9   transactions              200 non-null    object 
 10  price                     112 non-null    object 
 11  phone                     200 non-null    object 
 12  display_phone             200 non-null    object 
 13  distance                  200 non-null    float64
 14  coordinate

Calculando y eliminando duplicados

In [12]:
rests.duplicated(['id']).sum()

np.int64(4)

In [13]:
rests.drop_duplicates(subset=['id'],inplace=True)
rests.duplicated(['id']).sum()
rests.info()

<class 'pandas.core.frame.DataFrame'>
Index: 196 entries, 0 to 199
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        196 non-null    object 
 1   alias                     196 non-null    object 
 2   name                      196 non-null    object 
 3   image_url                 196 non-null    object 
 4   is_closed                 196 non-null    bool   
 5   url                       196 non-null    object 
 6   review_count              196 non-null    int64  
 7   categories                196 non-null    object 
 8   rating                    196 non-null    float64
 9   transactions              196 non-null    object 
 10  price                     110 non-null    object 
 11  phone                     196 non-null    object 
 12  display_phone             196 non-null    object 
 13  distance                  196 non-null    float64
 14  coordinates_lat

Calculando y eliminando nulos

In [14]:
rests['price'].isna().sum()

np.int64(86)

In [15]:
rests['price'] = rests['price'].fillna(rests['price'].mode()[0])
rests['price'].isna().sum()
rests.info()

<class 'pandas.core.frame.DataFrame'>
Index: 196 entries, 0 to 199
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        196 non-null    object 
 1   alias                     196 non-null    object 
 2   name                      196 non-null    object 
 3   image_url                 196 non-null    object 
 4   is_closed                 196 non-null    bool   
 5   url                       196 non-null    object 
 6   review_count              196 non-null    int64  
 7   categories                196 non-null    object 
 8   rating                    196 non-null    float64
 9   transactions              196 non-null    object 
 10  price                     196 non-null    object 
 11  phone                     196 non-null    object 
 12  display_phone             196 non-null    object 
 13  distance                  196 non-null    float64
 14  coordinates_lat

Eliminación de columnas

In [16]:
## Existen columnas que no aportan valor al análisis, por lo que decidimos eliminarlas.

rests = rests.drop(['location_address1', 'location_address2','location_address3','location_city','location_zip_code','location_country','location_state','location_display_address'], axis=1)
rests.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,phone,display_phone,distance,coordinates_latitude,coordinates_longitude,city
0,8YWLuLUKj0t_0_Xv06UUtw,yes-apothecary-new-york-6,Ye's Apothecary,https://s3-media0.fl.yelpcdn.com/bphoto/y4pAeB...,False,https://www.yelp.com/biz/yes-apothecary-new-yo...,998,"[{'alias': 'szechuan', 'title': 'Szechuan'}, {...",4.6,[],$$,+19296552515,(929) 655-2515,1591.379869,40.719450,-73.989800,New York
1,DEp-9JVnKN7eTb9A_mUlfA,zou-zou-s-new-york-3,Zou Zou’s,https://s3-media0.fl.yelpcdn.com/bphoto/JFsmaq...,False,https://www.yelp.com/biz/zou-zou-s-new-york-3?...,536,"[{'alias': 'mediterranean', 'title': 'Mediterr...",4.2,[],$$$,+12123808585,(212) 380-8585,5290.268089,40.753026,-73.998240,New York
2,dEOv8_ivdHp85OK_TDQh_g,da-andrea-chelsea-new-york-3,Da Andrea - Chelsea,https://s3-media0.fl.yelpcdn.com/bphoto/8P1kzX...,False,https://www.yelp.com/biz/da-andrea-chelsea-new...,656,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.7,"[pickup, restaurant_reservation, delivery]",$$$,+12123545971,(212) 354-5971,4090.350456,40.742041,-74.000467,New York
3,ysqgdbSrezXgVwER2kQWKA,julianas-brooklyn-3,Juliana's,https://s3-media0.fl.yelpcdn.com/bphoto/od36nF...,False,https://www.yelp.com/biz/julianas-brooklyn-3?a...,2928,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.4,[delivery],$$,+17185966700,(718) 596-6700,318.876261,40.702747,-73.993435,New York
4,-_J1rWztwR5fXFSqC6yPDg,elephant-district-brooklyn,Elephant District,https://s3-media0.fl.yelpcdn.com/bphoto/3VPck8...,False,https://www.yelp.com/biz/elephant-district-bro...,124,"[{'alias': 'thai', 'title': 'Thai'}]",4.6,"[pickup, delivery]",$$,+18453143606,(845) 314-3606,289.226892,40.702952,-73.993766,New York


Columnas numericas y seleccion de columnas para tabla

In [17]:
## Aseguramos que las columnas sean numéricas

rests['rating'] = pd.to_numeric(rests['rating'], errors='coerce')
rests['review_count'] = pd.to_numeric(rests['review_count'], errors='coerce')

c = rests['rating'].mean()

m = rests['review_count'].quantile(0.75) ## se toma el 25% en numeros de reviews

def weighted_rating(x, m=m, c=c):
    v = x['review_count']
    R = x['rating']
    # Calculamos el weighted rating
    return (v/(v+m) * R) + (m/(m+v) * c)


Guardando en formato .CSV

In [18]:
#Seleccción de columnas para nueva tabla que muestra los restaurantes con mejor score.
rests['score'] = rests.apply(weighted_rating, axis=1)
mejor_restaurante = rests.sort_values('score', ascending=False)
columnas_interes = ['name', 'city', 'review_count', 'rating', 'price', 'score']
mejor_restaurante = mejor_restaurante[columnas_interes]
mejor_restaurante.head(20)

,name,city,review_count,rating,price,score
42,Los Tacos No.1,New York,3415,4.7,$$,4.652801
2,Da Andrea - Chelsea,New York,656,4.7,$$$,4.546563
158,L'industrie Pizzeria,New York,1683,4.6,$$,4.541804
0,Ye's Apothecary,New York,998,4.6,$$,4.516377
36,CHILI,New York,930,4.6,$$,4.512585
17,Gurumé,New York,320,4.7,$$,4.488620
115,Taste of Heaven,New York,317,4.7,$$,4.487905
116,Bolivian Llama Party,New York,291,4.7,$$,4.481498
133,THEP Thai Restaurant,New York,3424,4.5,$$$,4.481440
46,Leitao,New York,259,4.7,$$$,4.473061


In [20]:
rests.to_csv('restaurantes_nyc.csv', index=True)
print(f"Se han guardado {len(rests)} resultados en 'restaurantes_nyc.csv'")

Se han guardado 196 resultados en 'restaurantes_nyc.csv'


----
